In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import xgboost as xgb
import seaborn as sns
import gc
import os
color = sns.color_palette()

%matplotlib inline

pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999
os.chdir('/san-data/usecase/magnet_g/misc/PCA_DATA_VIS/Yiming/Summer 2017') 

In [2]:
print('Loading data ...')

train = pd.read_csv('train_2016_v2.csv')
prop = pd.read_csv('properties_2016.csv')

Loading data ...


/opt/anaconda/4.1.1/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Convert variable type from float64 to float32.

In [3]:
for c, dtype in zip(prop.columns, prop.dtypes):
    if dtype == np.float64:
        prop[c] = prop[c].astype(np.float32)

Now let's creating training set by merging two datasets together by "parcelid".

In [4]:
df_train = train.merge(prop, how = "left", on = "parcelid")
mean_values = df_train.mean(axis = 0)
df_train = df_train.fillna(mean_values, inplace = True) # imputation

# drop some unnecessary columns
# predictors
x_train = df_train.drop(['parcelid', 'logerror', 'transactiondate', 'propertyzoningdesc', 'propertycountylandusecode'], axis = 1)

# target
y_train = df_train['logerror'].values

# shapes
print(x_train.shape, y_train.shape)

((90275, 55), (90275,))


In [5]:
# train columns (variable names)
train_columns = x_train.columns

for c in x_train.dtypes[x_train.dtypes == object].index.values:
    x_train[c] = (x_train[c] == True)

del df_train

gc.collect()

7

### Train-valid Split
Training set is the first 80000 rows, validation set is the remaining rows.

In [6]:
split = 80000
x_train, y_train, x_valid, y_valid = x_train[:split], y_train[:split], x_train[split:], y_train[split:]

### Train the model

In [7]:
d_train = xgb.DMatrix(x_train, label = y_train)
d_valid = xgb.DMatrix(x_valid, label = y_valid)

params = {
    'eta': 0.02, 
    'max_depth': 4,
    'objective': 'reg:linear',
    'silent': 1
}

watchlist = [(d_train, 'train'), (d_valid, 'valid')]
clf = xgb.train(params, d_train, 10000, watchlist, early_stopping_rounds = 100, verbose_eval = 10)

Will train until valid error hasn't decreased in 100 rounds.
[0]	train-rmse:0.506043	valid-rmse:0.497858
[10]	train-rmse:0.423753	valid-rmse:0.415290
[20]	train-rmse:0.358409	valid-rmse:0.349657
[30]	train-rmse:0.307110	valid-rmse:0.298022
[40]	train-rmse:0.267432	valid-rmse:0.257970
[50]	train-rmse:0.237248	valid-rmse:0.227515
[60]	train-rmse:0.214724	valid-rmse:0.204738
[70]	train-rmse:0.198238	valid-rmse:0.188093
[80]	train-rmse:0.186330	valid-rmse:0.176197
[90]	train-rmse:0.177901	valid-rmse:0.167854
[100]	train-rmse:0.171964	valid-rmse:0.162076
[110]	train-rmse:0.167831	valid-rmse:0.158110
[120]	train-rmse:0.164977	valid-rmse:0.155456
[130]	train-rmse:0.163009	valid-rmse:0.153603
[140]	train-rmse:0.161637	valid-rmse:0.152328
[150]	train-rmse:0.160650	valid-rmse:0.151472
[160]	train-rmse:0.159974	valid-rmse:0.150991
[170]	train-rmse:0.159466	valid-rmse:0.150678
[180]	train-rmse:0.159079	valid-rmse:0.150443
[190]	train-rmse:0.158806	valid-rmse:0.150326
[200]	train-rmse:0.158552	vali

In [23]:
# Build test set
sample = pd.read_csv('sample_submission.csv')
sample['parcelid'] = sample['ParcelId']
df_test = sample.merge(prop, on = 'parcelid', how = 'left')

In [24]:
mean_values = df_test[train_columns].mean(axis = 0)
df_test[train_columns] = df_test[train_columns].fillna(mean_values, inplace = True) # imputation

In [25]:
x_test = df_test[train_columns]
for c in x_test.dtypes[x_test.dtypes == object].index.values:
    x_test[c] = (x_test[c] == True)
    
d_test = xgb.DMatrix(x_test)

### Predictions

In [26]:
p_test = clf.predict(d_test)

sub = pd.read_csv('sample_submission.csv')
for c in sub.columns[sub.columns != 'ParcelId']:
    sub[c] = p_test
    
# write to csv, 4 decimals
sub.to_csv('xgb_starter.csv', index = False, float_format = '%.4f')

In [27]:
sub.shape

(2985217, 7)